In [1]:
import numpy as np

In [2]:
def import_txt(file_str):
    '''Read files as hex arrays. Returns an array of tracks with 28 hex strings per track.
    First 12 hex strings are track fit constants, and last 16 hex strings are hit coordinates.'''
    results = []
    with open(file_str) as inputfile:
        for line in inputfile:
            results.append(line.strip()[-4:].split(','))
    track_results = results[14:len(results)-16]
    n_track = int(np.floor(len(track_results)/28))
    track_v = np.array(track_results).reshape(n_track,-1)
    return track_v

In [3]:
track_v = import_txt('ftksim_3tracks.txt')
hw_v = import_txt('ftksim_3tracks_hwout.txt')

In [76]:
def matching_track_clusters(track_v):
    '''Given an array of tracks, return a list of matching (sharing 6 or more hits) track indices.
    e.g. Given 5 tracks, where tracks 0 and 1 match, and tracks 3 and 4 match, return [[0,1], [2], [3,4]].
    Note that if tracks 0 and 1 match, and tracks 1 and 2 match, we consider tracks 0 and 2 to match.'''
    def n_matching_hits(track_i, track_j):
        '''Number of matching hits (IBL+SCT) between two tracks.'''
        matches = (track_i[12:] == track_j[12:])
        n_IBL_matches = (matches[0]&matches[1])+(matches[2]&matches[3])+(matches[4]&matches[5])+(matches[6]&matches[7])
        n_SCT_matches = sum(matches[8:16])
        return n_IBL_matches + n_SCT_matches
    track_clusters = []
    for i in range(len(track_v)):
        current_cluster = []
        for j in range(i, len(track_v)):
            if n_matching_hits(track_v[i], track_v[j])>6: current_cluster.append(j)
        is_new_cluster = True
        for n, cluster in enumerate(track_clusters):
            combined_cluster = set(cluster+current_cluster)
            if len(combined_cluster) < len(cluster) + len(current_cluster):
                track_clusters[n] = list(combined_cluster)
                is_new_cluster = False
                break
        if is_new_cluster: track_clusters.append(current_cluster)
    track_clusters = [np.array(i) for i in track_clusters]
    return track_clusters

def hex2bin(hex_num):
    '''Replaces each character of a hex string (ignoring the leading 0) with four bits.
    e.g. hex2bin("0af1") = "101011110001".'''
    def hex2bin_map(x):
        '''Interprets a character as hex and converts to a padded 4-digit binary string.'''
        return bin(int("0x"+x,16))[2:].zfill(4)
    bin_num = ""
    for i in range(1,4):
        bin_num = bin_num+hex2bin_map(hex_num[i])
    return bin_num

def track_picking(rc_v, track_v):    
    def layer_map_sum(layer_map):
        '''Sums up the number of 1\'s in a string.
        e.g. "111101110111" returns 11.'''
        return layer_map.count("1")
    if isinstance(rc_v, np.ndarray):
        sum_v = np.zeros(len(rc_v))
        chisq_v = []
        for i in range(len(rc_v)):
            sum_v[i] = layer_map_sum(hex2bin(track_v[rc_v[i],3]))
            chisq_v.append(track_v[rc_v[i],6])
        l_max = np.where(sum_v == max(sum_v))[0]
        select_index = l_max[0]
        if len(l_max)>1:
            chisq_min = '7c00'
            #chisq_min = chisq_v[0]
            for j in range(len(l_max)):
                if chisq_min > chisq_v[l_max[j]]:
                    chisq_min = chisq_v[l_max[j]]
            select_index = chisq_v.index(chisq_min)
        track_index = rc_v[select_index]
    else: 
        track_index = rc_v
    ##print(track_index)
    return track_index

def HitWarrior(track_v):
    re_cluster = matching_track_clusters(track_v)
    HW_output_index = np.ones((len(re_cluster),), dtype=int)*200
    for i in range(len(re_cluster)):
        HW_output_index[i] = track_picking(re_cluster[i], track_v)
    print("")
    print(HW_output_index)
    return re_cluster, HW_output_index

In [77]:
rc, HW_out = HitWarrior(track_v)
#print(list(HW_out))
#print(list(rc))


[16  1 18  5  8 25]


In [6]:
for i in range(len(HW_out)):
    for j in range(len(hw_v)):
        if sum(track_v[HW_out[i], 12:28] == hw_v[j, 12:28]) > 10:
            print("in_track #",HW_out[i]," --> out_track #",j, sep="")
            print(track_v[HW_out[i]] == hw_v[j])

hw_v[0]
hw_v[0] == track_v[25]
hw_v[5] == track_v[8]
hw_v[1] == track_v[16]
hw_v[2] == track_v[18]
hw_v[3] == track_v[1]
hw_v[4] == track_v[5]
list(hw_v[2])

for i in range(28):
    if i == 12:
        print("----------------------------------------------")
    print(hw_v[:,i])

for i in range(28):
    print(hw_v[2,i], track_v[2,i], track_v[3,i], track_v[4,i], track_v[17,i], track_v[18,i])

print(hex2bin(track_v[0,3]),hex2bin(track_v[6,3]),hex2bin(track_v[7,3]),hex2bin(track_v[9,3]),hex2bin(track_v[12,3]),hex2bin(track_v[16,3]))
print(hex2bin(track_v[8,3]),hex2bin(track_v[10,3]),hex2bin(track_v[11,3]),hex2bin(track_v[13,3]),hex2bin(track_v[14,3]))

track_v[16]
hw_v[1]
track_v[8]
track_v[16]==hw_v[1]

hex_x = np.array([0x420c])
hex_x.astype(float)

test_list = [track_v[15,6],track_v[15,6],track_v[19,6],track_v[23,6],track_v[24,6],track_v[25,6],track_v[26,6]]
test_v = np.array(test_list)
test_v

np.argmin([track_v[8,6],track_v[10,6],track_v[11,6],track_v[13,6],track_v[14,6]])
np.where(test_v == min(test_v))
min(test_v)
np.where(test_v == min(test_v))[0]

[track_v[8,5],track_v[10,5],track_v[11,5],track_v[13,5],track_v[14,5]]
[track_v[8,4],track_v[10,4],track_v[11,4],track_v[13,4],track_v[14,4]]
track_v[8]
hw_v[5]==track_v[8]

in_track #16 --> out_track #1
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
in_track #1 --> out_track #3
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
in_track #18 --> out_track #2
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
in_track #5 --> out_track #4
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
in_track #8 --> out_track #5
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True 

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])